In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from datasets import Dataset
import os

In [1]:
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"

NameError: name 'os' is not defined

In [ ]:
model_pt = "google/gemma-3-1b-pt"
model_ft = "models/gemma-3-1b-ft"
log_dir = "models/logs"
tokenizer = AutoTokenizer.from_pretrained(model_pt)
model = AutoModelForCausalLM.from_pretrained(model_pt)

# Force CPU usage
device = torch.device("cpu")
model.to(device)


ValueError: The checkpoint you are trying to load has model type `gemma3_text` but Transformers does not recognize this architecture. This could be because of an issue with the checkpoint, or because your version of Transformers is out of date.

You can update Transformers with the command `pip install --upgrade transformers`. If this does not work, and the checkpoint is very new, then there may not be a release version that supports this model yet. In this case, you can get the most up-to-date code by installing Transformers from source with the command `pip install git+https://github.com/huggingface/transformers.git`

In [ ]:
dummy_data = [
    {"prompt": "What is the capital of France?", "answer": "The capital of France is Paris."},
    {"prompt": "Who wrote Hamlet?", "answer": "Hamlet was written by William Shakespeare."}
]

# Function to format data for training
def format_conversation(example):
    return {"text": f"Prompt: {example['prompt']}\nAnswer: {example['answer']}"}

formatted_data = [format_conversation(d) for d in dummy_data]
dataset = Dataset.from_list(formatted_data)

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(["text"])
tokenized_dataset.set_format("torch")

In [ ]:
trainer = Trainer(
    model=model,
    args=TrainingArguments(
    output_dir=model_ft,
    per_device_train_batch_size=1,
    num_train_epochs=1,
    save_strategy="epoch",
    logging_dir=log_dir,
    logging_steps=1,
    save_total_limit=2,
    evaluation_strategy="no",
    report_to="none",  # Disable wandb logging
),
    train_dataset=tokenized_dataset,
)

trainer.train()

In [ ]:
model.save_pretrained(model_ft)
tokenizer.save_pretrained(model_ft)